In [3]:
import glob
import os
import pandas as pd
import csv

In [4]:
path = '/Users/dominicgraziano/Documents/Bricklink_Orders/Data/'

Get all of the csv files within my data folder

In [5]:
files = glob.glob(os.path.join(path, '*csv')) 
files = files

Takes all of the csv files in the list and concatenates them into a single dataframe

In [6]:
combine_df = pd.DataFrame()
for file in files:
    df = pd.read_csv(file)
    combine_df = pd.concat([combine_df,df])

In [7]:
combine_df.columns

Index(['Order ID', 'Order Date', 'Buyer', 'Base Currency', 'Shipping',
       'Insurance', 'Add Chrg 1', 'Add Chrg 2', 'Credit', 'Coupon Credit',
       'Order Total', 'Tax', 'Base Grand Total', 'Total Lots', 'Total Items',
       'Order Status', 'Order Status Changed', 'Pmt In', 'Pmt Method',
       'Order Note', 'Tracking No', 'Location', 'Batch', 'Batch Date',
       'Condition', 'Item Description', 'My Remarks', 'Qty', 'Each', 'Total',
       'Item Type', 'Item Number', 'Weight', 'Inv ID', 'Sub-Condition',
       'Unnamed: 35', 'Unnamed: 36'],
      dtype='object')

Filtering out unneccessary columns to get those that will be uploaded into the database

In [8]:
columns = ['Order ID', 'Order Date', 'Buyer', 'Base Currency', 'Shipping',
                'Order Total', 'Tax', 'Base Grand Total', 'Total Lots', 'Total Items',
                'Order Status','Location','Condition', 'Item Description','Qty', 'Each',
                'Total','Item Type', 'Item Number','Inv ID']

end_df = combine_df[columns]
end_df = end_df[1:]
print(len(end_df.columns))

KeyError: 'Cannot get left slice bound for non-unique label: 1'

Changing the datetime to a format that will be accepted by MySQL

In [ ]:
end_df['Order Date']= pd.to_datetime(end_df['Order Date'],format = '%m/%d/%Y').dt.strftime('%Y-%m-%d')

Remove the $ from all of the prices, so they will be accepted as floats

In [ ]:
end_df = end_df.applymap(lambda x: x.replace('$', '') if isinstance(x, str) else x)


In [ ]:
end_df['Item Number'] = end_df['Item Number'].fillna('11')
end_df['Item Number'] = end_df['Item Number'].astype(str)

In [ ]:
end_df.head()

,Order ID,Order Date,Buyer,Base Currency,Shipping,Order Total,Tax,Base Grand Total,Total Lots,Total Items,Order Status,Location,Condition,Item Description,Qty,Each,Total,Item Type,Item Number,Inv ID
1,23214652.0,2023-09-29,ArcStore27,USD,4.99,6.50,0.47,11.96,2.0,2.0,Shipped,"USA, California",NaN,NaN,NaN,NaN,NaN,NaN,11,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Used,Trans-Light Blue Brick 1 x 2 x 5 without Side ...,1.0,3.50,3.50,Part,46212pb001,366406343.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Used,"White Minifigure, Headgear Helmet SW Stormtroo...",1.0,3.00,3.00,Part,30408p01,366416338.0
4,23170630.0,2023-09-23,hanlike,USD,4.99,31.94,2.87,39.80,6.0,11.0,Shipped,"USA, California",NaN,NaN,NaN,NaN,NaN,NaN,11,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Used,Rene Belloq - Dark Tan Jacket,1.0,9.48,9.48,Minifigure,iaj009,363929481.0


Created a new csv which will be the one used to load the data 

In [ ]:
end_df.to_csv('Bricklink_Sales.csv',index=False)

In [ ]:
with open('Bricklink_Sales.csv', 'r') as csv_file:
   csv_data = list(csv.reader(csv_file))

In [ ]:
import mysql.connector

Use the MySQL connector to connect to my local host

In [ ]:
db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    passwd = password
)
mycursor = db.cursor()

In [ ]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS Bricklink_Sales")

In [ ]:
db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    passwd = password,
    database = 'Bricklink_Sales'
)
mycursor = db.cursor()

Columns we want 
['Order ID', 'Order Date', 'Buyer', 'Base Currency', 'Shipping',
                'Order Total', 'Tax', 'Base Grand Total', 'Total Lots', 'Total Items',
                'Order Status','Location','Condition', 'Item Description','Qty', 'Each',
                'Total','Item Type', 'Item Number','Inv ID']

Execute creates the table within the database, while specifiying each columnn name and datatype to be acceptedd

In [ ]:
mycursor.execute('CREATE TABLE IF NOT EXISTS Sales (order_id INT(10), order_date DATE, buyer VARCHAR(50), base_currency VARCHAR(10), shipping FLOAT(20), order_total FLOAT(10), tax FLOAT(10), base_grand_total FLOAT(10),total_lots INT(10), total_items INT(10),order_status VARCHAR(50),location VARCHAR(50),item_condition VARCHAR(50),item_description VARCHAR(200),quantity INT(10),each_price FLOAT(10),total FLOAT(10),item_type VARCHAR(50),item_number VARCHAR(50),invoice_id INT(10))')

for each row it goes across the columns and adds to each of teh specified columns, once closed we can access the database outside of Python as well

In [ ]:
with open('Bricklink_Sales.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    # Assuming the first row contains column headers, skip it
    next(csv_reader)
    for row in csv_reader:
        # Replace empty or missing values with None
        for i in range(len(row)):
            if row[i] == '':
                row[i] = None
                
       
        # SQL INSERT statement with placeholders
        sql = """
            INSERT INTO Sales (
                order_id, order_date, buyer, base_currency, shipping, order_total,
                tax, base_grand_total, total_lots, total_items, order_status,
                location, item_condition, item_description, quantity, each_price,
                total, item_type, item_number, invoice_id
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Execute the SQL statement with the row data
       

        mycursor.execute(sql, tuple(row))
    db.commit()
mycursor.close()
db.close()